In [73]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('C:\\Users\\37424\\Desktop\\fake_transactional_data_24.csv')
unique_users = df['from_totally_fake_account'].nunique()
unique_merchants = df['to_randomly_generated_account'].nunique()
print(f'User: {unique_users}')
print(f'Merchants: {unique_merchants}')

User: 8142
Merchants: 14300


In [74]:
df['from_totally_fake_account'] = df['from_totally_fake_account'].astype(int)
df['not_happened_yet_date'] = pd.to_datetime(df['not_happened_yet_date'], format='%d/%m/%Y')

In [75]:
grouped = df.groupby('from_totally_fake_account')
sorted_users = [group.sort_values('not_happened_yet_date') for _, group in grouped]
sorted_df = pd.concat(sorted_users)
aa = sorted_df['from_totally_fake_account'].count()
print(aa)
print(sorted_df.head(100).to_string(index=False))
sorted_df.to_csv('users_sorted_dataframe.csv', index=False)

10148280
 from_totally_fake_account  monopoly_money_amount to_randomly_generated_account not_happened_yet_date
                      1000                   8.99             STREAMING_SERVICE            2025-01-01
                      1000                   2.40           GOURMET_COFFEE_SHOP            2025-01-01
                      1000                   2.55           GOURMET_COFFEE_SHOP            2025-01-01
                      1000                   2.20           A_LOCAL_COFFEE_SHOP            2025-01-01
                      1000                  13.06                     LOCAL_PUB            2025-01-01
                      1000                   4.50                          CAFE            2025-01-02
                      1000                   2.45           HIPSTER_COFFEE_SHOP            2025-01-02
                      1000                   2.20           GOURMET_COFFEE_SHOP            2025-01-02
                      1000                   2.65       PRETENTIOUS_COFFE

In [76]:
df['month'] = df['not_happened_yet_date'].dt.month
monthly_expenses = df.groupby(['from_totally_fake_account', 'month'])['monopoly_money_amount'].sum().reset_index()

monthly_expenses.rename(columns={'monopoly_money_amount': 'total_monthly_spent'}, inplace=True)

print(monthly_expenses.head(100).to_string(index=False))
monthly_expenses.to_csv('monthly_expenses.csv', index=False)

 from_totally_fake_account  month  total_monthly_spent
                      1000      1              2576.52
                      1000      2              2882.33
                      1000      3              2341.74
                      1000      4              1876.77
                      1000      5              1843.29
                      1000      6              2553.40
                      1000      7              2949.72
                      1000      8              2503.66
                      1000      9              2173.79
                      1000     10              2502.73
                      1000     11              1882.36
                      1000     12              2612.20
                      1002      1              1097.19
                      1002      2              1412.92
                      1002      3              1263.27
                      1002      4              1132.20
                      1002      5              1520.16
          

In [77]:
df['year'] = df['not_happened_yet_date'].dt.year
year_expenses = df.groupby(['from_totally_fake_account', 'year'])['monopoly_money_amount'].sum().reset_index()

year_expenses.rename(columns={'monopoly_money_amount': 'total_year_spent'}, inplace=True)

print(year_expenses.head(100).to_string(index=False))
year_expenses.to_csv('year_expenses.csv', index=False)

 from_totally_fake_account  year  total_year_spent
                      1000  2025          28698.51
                      1002  2025          15622.97
                      1006  2025          18778.19
                      1018  2025          15612.21
                      1053  2025          19444.57
                      1059  2025          17491.45
                      1071  2025          13505.42
                      1078  2025          14967.27
                      1086  2025          20213.62
                      1087  2025          15920.14
                      1113  2025          12201.90
                      1120  2025          18942.52
                      1122  2025          30817.08
                      1125  2025          14925.60
                      1133  2025          23130.41
                      1142  2025          15215.22
                      1174  2025          15780.56
                      1179  2025          30291.95
                      1180  202

In [78]:
#RFM
current_date = df['not_happened_yet_date'].max()
rfm = df.groupby('from_totally_fake_account').agg({
    'not_happened_yet_date': lambda x: (current_date - x.max()).days,  # Recency
    'to_randomly_generated_account': 'count',                              # Frequency
    'monopoly_money_amount': 'sum'                                     # Monetary
}).reset_index().rename(columns={
    'not_happened_yet_date': 'recency',
    'to_randomly_generated_account': 'frequency',
    'monopoly_money_amount': 'monetary'
})

quantiles = rfm.quantile(q=[0.25, 0.5, 0.75])
def recency_score(x, bins):
    if x == bins[0]:
        return 1
    elif x <= bins[1]:
        return 2
    elif x <= bins[2]:
        return 3
    else:
        return 4
recency_bins = [0, 2, 4]
rfm['r_score'] = rfm['recency'].apply(recency_score, bins=recency_bins)    #The smaller the value, the closer the time is
rfm['f_score'] = pd.qcut(rfm['frequency'], q=4, labels=range(1, 5))
rfm['m_score'] = pd.qcut(rfm['monetary'], q=4, labels=range(1, 5))

print(rfm.to_string(index=False))
rfm.to_csv('rfm_for_all_user.csv', index=False)

 from_totally_fake_account  recency  frequency  monetary  r_score f_score m_score
                      1000        0       1710  28698.51        1       4       3
                      1002        0       1043  15622.97        1       2       1
                      1006        0       1841  18778.19        1       4       2
                      1018        1        805  15612.21        2       1       1
                      1053        0       1319  19444.57        1       3       2
                      1059        0       1004  17491.45        1       2       1
                      1071        0        930  13505.42        1       1       1
                      1078        0        958  14967.27        1       1       1
                      1086        0       1210  20213.62        1       3       2
                      1087        0       1703  15920.14        1       4       1
                      1113        0        983  12201.90        1       1       1
                

In [79]:
grouped = df.groupby('to_randomly_generated_account')
sorted_merchants = [group.sort_values('not_happened_yet_date') for _, group in grouped]
sorted_df2 = pd.concat(sorted_merchants)
print(sorted_df2.head(1000).to_string(index=False))

 from_totally_fake_account  monopoly_money_amount to_randomly_generated_account not_happened_yet_date  month  year
                      5403                   6.00                          1000            2025-01-04      1  2025
                      5403                   6.00                          1000            2025-01-04      1  2025
                      5403                   6.00                          1000            2025-01-04      1  2025
                     77084                   4.50                          1000            2025-01-05      1  2025
                     77084                   4.50                          1000            2025-01-05      1  2025
                     77084                 334.00                          1000            2025-01-07      1  2025
                     56140                 334.00                          1000            2025-01-07      1  2025
                     41718                   5.50                          1000 

In [80]:
columns = list(df.columns)
columns.remove('to_randomly_generated_account')
new_columns_order = ['to_randomly_generated_account'] + columns
df = df[new_columns_order]
df_filtered = df[~df['to_randomly_generated_account'].str.isdigit()]
# print(df_filtered.head(1000).to_string(index=False))
grouped = df_filtered.groupby('to_randomly_generated_account')
sorted_merchants = [group.sort_values('not_happened_yet_date') for _, group in grouped]
sorted_df2 = pd.concat(sorted_merchants)
aa = sorted_df2['from_totally_fake_account'].count()
print(f'Records: {aa}')
unique_merchants = sorted_df['to_randomly_generated_account'].nunique()
print(f'Merchants: {unique_merchants}')
print(sorted_df2.head(100).to_string(index=False))
sorted_df2.to_csv('merchants_sorted_dataframe.csv', index=False)

Records: 7615517
Merchants: 14300
to_randomly_generated_account  from_totally_fake_account  monopoly_money_amount not_happened_yet_date  month  year
               ACCESSORY_SHOP                      95445                   18.0            2025-01-01      1  2025
               ACCESSORY_SHOP                      10179                   14.0            2025-01-01      1  2025
               ACCESSORY_SHOP                      63705                   18.0            2025-01-01      1  2025
               ACCESSORY_SHOP                      79889                   39.0            2025-01-01      1  2025
               ACCESSORY_SHOP                      52591                   39.0            2025-01-01      1  2025
               ACCESSORY_SHOP                      26805                   12.0            2025-01-01      1  2025
               ACCESSORY_SHOP                      19533                   29.0            2025-01-02      1  2025
               ACCESSORY_SHOP                 

In [81]:
sorted_df2['month'] = sorted_df2['not_happened_yet_date'].dt.month
monthly_incomes = sorted_df2.groupby(['to_randomly_generated_account', 'month'])['monopoly_money_amount'].sum().reset_index()

monthly_incomes.rename(columns={'monopoly_money_amount': 'total_monthly_income'}, inplace=True)
print(monthly_incomes.head(100).to_string(index=False))
monthly_incomes.to_csv('monthly_incomes.csv', index=False)

to_randomly_generated_account  month  total_monthly_income
               ACCESSORY_SHOP      1               6238.00
               ACCESSORY_SHOP      2               6213.00
               ACCESSORY_SHOP      3               6545.00
               ACCESSORY_SHOP      4               5867.00
               ACCESSORY_SHOP      5               6447.00
               ACCESSORY_SHOP      6               6553.00
               ACCESSORY_SHOP      7               6481.00
               ACCESSORY_SHOP      8               6585.00
               ACCESSORY_SHOP      9               5778.00
               ACCESSORY_SHOP     10               6186.00
               ACCESSORY_SHOP     11               5949.00
               ACCESSORY_SHOP     12               6020.00
                       A_CAFE      1             100404.55
                       A_CAFE      2              88050.80
                       A_CAFE      3              93726.05
                       A_CAFE      4              95313.

In [82]:
sorted_df2['year'] = sorted_df2['not_happened_yet_date'].dt.year
year_incomes = sorted_df2.groupby(['to_randomly_generated_account', 'year'])['monopoly_money_amount'].sum().reset_index()

year_incomes.rename(columns={'monopoly_money_amount': 'total_year_incomes'}, inplace=True)

print(year_incomes.head(100).to_string(index=False))
year_expenses.to_csv('year_incomes.csv', index=False)

to_randomly_generated_account  year  total_year_incomes
               ACCESSORY_SHOP  2025            74862.00
                       A_CAFE  2025          1144734.25
          A_LOCAL_COFFEE_SHOP  2025          1141932.30
                A_SUPERMARKET  2025          5652778.58
                          BAR  2025          8458081.50
                     BOOKSHOP  2025           324530.00
                      BUTCHER  2025          2097495.50
                     BUTCHERS  2025           294408.00
                         CAFE  2025           489245.50
              CHILDRENDS_SHOP  2025           916095.00
           CHINESE_RESTAURANT  2025           262599.78
             CHINESE_TAKEAWAY  2025          2140249.81
                       CINEMA  2025           433414.00
                 CLOTHES_SHOP  2025          2828740.00
                 COCKTAIL_BAR  2025          1366895.50
                  COFFEE_SHOP  2025          1146046.25
              COMIC_BOOK_SHOP  2025           26